In [1406]:
from subprocess import getoutput
import os
import re 
import json
#%pip install libgraphviz-dev
#%pip install pygraphviz
#%pip install graphviz
import graphviz
from collections import defaultdict

In [1407]:
def ValidateRegex (RegexInput):
  try:
    re.compile(RegexInput)
    return True

  except re.error:
    print("Non valid regex pattern")
    exit()

In [1408]:
def ExtractAtomicExpressions(regex):
    stack = list() 
    atomic = str()
    atomics = list()
    for c in regex:
        if c != ")":
            stack.append(c)
        else:
            while stack[-1] != "(" and len(stack) > 0 : #Extract the atomic expression that corresponds to the closing paranthesis found
                atomic = stack[-1] + atomic
                stack.pop(-1)
            
            stack.pop(-1) #remove the opening bracket from the stack
            atomics.append(atomic)
            atomic = ""
    #atomics.reverse()
    return atomics

In [1409]:
class State:
    stateID=None
    edges=[]
    transitions=[]    
    def __init__(self, stateID=None, edges=None,transitions=None):
        self.stateID = stateID
        self.edges = edges or []

In [1410]:
class NFA:
    initial = None
    accept = None

    def __init__(self, initial, accept):
        self.initial = initial
        self.accept = accept or []

In [1411]:

def InsertConcatenationDot(regex):
    #regex = re.sub(r'\)(\w|\()', r').\1', regex)
    regex = re.sub(r'\)(\()', r').\1', regex)
    regex = re.sub(r'(\w|\))(\(|\w)', r'\1.\2', regex)
    #regex = re.sub(r'\](?=[a-zA-Z0-9])', '].', regex)
    #regex = re.sub(r'\)(?=[a-zA-Z0-9])', ').', regex)
    #regex = re.sub(r'(\w|\))\(', r'\1.(', regex)
    #regex = re.sub(r'\)([A-Z123]\()', ').\g<1>', regex)
    #regex = re.sub(r'\)', ').', regex)


    return regex



In [1412]:
#regex=("((xy)z)")
#regex=("(xy)")
regex="(([A-Z123])([drytf56876]))"
#regex = re.sub(r'(\w|\))\(', r'\1.(', regex)
print(InsertConcatenationDot(regex))
r = InsertConcatenationDot(regex)
print(ExtractAtomicExpressions(r))

(([A-Z.12.3]).([d.ry.tf.56.87.6]))
['[A-Z.12.3]', '[d.ry.tf.56.87.6]', '.']


In [1413]:
def MoveSymbolToEndOfAtomicExpression(expression):
    for i in range(len(expression)):
        if expression[i] == "|":
            return expression[0:i] + expression[i+1:] + "|"
        if expression[i] == ".":
            return expression[0:i] + expression[i+1:] + "."
    return expression
            

In [1414]:
def RegexToNFA(regex):   
    if ValidateRegex(regex):
        regex = InsertConcatenationDot(regex)
        regex = regex.replace(" ", "")
        atomics = ExtractAtomicExpressions(regex)
        states = list()
        for i in range(len(atomics)):
            atomics[i] = MoveSymbolToEndOfAtomicExpression(atomics[i])
        counter = -1
        stack = []
        for atomic in atomics:
                match = re.search(r'\[.*?\]', atomic)
                if match:
                    character_class = match.group()
                    counter=counter+1
                    initial = State()
                    initial.stateID=counter
                    accept = State()
                    accept.stateID=counter+1
                    initial.edges.append((character_class, accept))
                    accept.edges=[]
                    stack.append(NFA(initial, [accept]))
                    counter=counter+1
                    continue
                
            #else:                
                for i in range(len(atomic)):
                    if atomic[i] == '.':
                        NFA2 = stack.pop()
                        NFA1 = stack.pop()
                        NFA1.accept[0].edges.append(("epsilon", NFA2.initial))
                        NFA1.accept = NFA2.accept
                        stack.append(NFA1)
                    elif atomic[i] == '|':
                        NFA2 = stack.pop()
                        NFA1 = stack.pop()
                        counter=counter+1
                        initial = State()
                        initial.stateID=counter
                        accept=State()
                        counter=counter+1
                        accept.stateID=counter
                        initial.edges.append(("epsilon", NFA1.initial))
                        initial.edges.append(("epsilon", NFA2.initial))
                        #accept = NFA1.accept + NFA2.accept
                        NFA1.accept[0].edges.append(("epsilon",accept))
                        NFA2.accept[0].edges.append(("epsilon",accept))
                        stack.append(NFA(initial, [accept]))
                    elif atomic[i] == '*':
                        counter=counter+1
                        NFA0 = stack.pop()
                        initial = State()
                        initial.stateID=counter
                        accept = State()
                        counter=counter+1
                        accept.stateID=counter
                        initial.edges.append(("epsilon", NFA0.initial))
                        initial.edges.append(("epsilon", accept))
                        NFA0.accept[0].edges.append(("epsilon", initial))
                        NFA0.accept[0].edges.append(("epsilon", accept))
                        stack.append(NFA(initial, [accept]))

                    
                    elif atomic[i] == '?':
                        counter=counter+1
                        NFA0 = stack.pop()
                        NFA0.initial.edges.append(("epsilon", NFA0.accept[0]))
                        stack.append(NFA0)

                    elif atomic[i] == '+':
                        counter=counter+1
                        NFA0 = stack.pop()
                        initial = State()
                        initial.stateID=counter
                        accept = State()
                        counter=counter+1
                        accept.stateID=counter
                        initial.edges.append(("epsilon", NFA0.initial))
                        NFA0.accept[0].edges.append(("epsilon", accept))
                        NFA0.accept[0].edges.append(("epsilon", NFA0.initial))
                        stack.append(NFA(initial, [accept]))

                
                    else:
                        counter=counter+1
                        initial = State()
                        initial.stateID=counter
                        accept = State()
                        accept.stateID=counter+1
                        initial.edges.append((atomic[i], accept))
                        stack.append(NFA(initial, [accept]))
                        counter=counter+1
    if len(stack) >= 1:
        return stack.pop()


In [1415]:
def WriteJsonFile(NFA):
 
    data = dict()
    visited = set()
    data["startingState"]="S"+str(NFA.initial.stateID)
    queue = [NFA.initial]
    while queue:
        state = queue.pop(0)
        if state in visited:
            continue
        visited.add(state)
        
        if state in NFA.accept:
                    
                    if data.get("S"+str(state.stateID)) is not None:
                        print("")
                    else:
                       data["S"+str(state.stateID)]=  dict()
                       data["S"+str(state.stateID)]["isTerminatingState"]="true"
                    for char, dest_state in state.edges:
                        if data.get("S"+str(state.stateID),{}).get(str(char)) is not None:
                            
                            data["S"+str(state.stateID)][str(char)].append("S"+str(dest_state.stateID))
                            
                        else:
                            data["S"+str(state.stateID)][str(char)]=list()
                            data["S"+str(state.stateID)][str(char)].append("S"+str(dest_state.stateID))

  
                        queue.append(dest_state)

                            
        for char, dest_state in state.edges:

            if data.get("S"+str(state.stateID)) is not None:
                print("")
            else:
                       data["S"+str(state.stateID)]= dict()
                       data["S"+str(state.stateID)]["isTerminatingState"]="false"
                       
            if data.get("S"+str(state.stateID))  is not None:
                if data.get("S"+str(state.stateID),{}).get(str(char))is not None:
                    data["S"+str(state.stateID)][str(char)].append("S"+str(dest_state.stateID))
                            
                else:
                    data["S"+str(state.stateID)][str(char)]=list()
                    data["S"+str(state.stateID)][str(char)].append("S"+str(dest_state.stateID))

            queue.append(dest_state)

    f = open('Json/NFA.json','w')
    json.dump(data,f)
    f.close()
    return data

In [1416]:
#regex= "((a(b*))c)"
#regex= "([a-c])"
#regex= "(AB)"
#regex= "(A*)"
#regex= "(A|B)"
regex="(([A-Z123])A)"
#regex= "(([A-Z123]*)|e)"
#regex="([A-Z123]*)"
#regex="(a?)"
#regex="(a+)"
#print(ExtractAtomicExpressions(regex))
r=InsertConcatenationDot(regex)
print(r)
print(ExtractAtomicExpressions(regex))
nfa=RegexToNFA(regex)
WriteJsonFile(nfa)

(([A-Z.12.3]).A)
['[A-Z123]', 'A']


{'startingState': 'S0',
 'S0': {'isTerminatingState': 'false', '[A-Z12.3]': ['S1']},
 'S1': {'isTerminatingState': 'false', 'epsilon': ['S2']},
 'S2': {'isTerminatingState': 'false', 'A': ['S3']},
 'S3': {'isTerminatingState': 'true'}}

In [1417]:
def draw_NFA(NFA):
    G = graphviz.Digraph()
    nodes = list(NFA.keys())[1:]
    for node in nodes:
        G.node(NFA.get("startingState"),color='green')
        if NFA.get(node).get('isTerminatingState') == 'true':
            G.node(node, shape='doublecircle' , color='red')
        else:
            G.node(node)
    for state in NFA:
        if state != "startingState":
            for transition in NFA.get(state):
                if transition != 'states' and transition != 'isTerminatingState':
                    for next_state in [NFA.get(state).get(transition)]:
                        for next_states in next_state:
                            if transition == "epsilon":
                                G.edge(state, next_states, label='ε')
                            #print("next state",next_state)
                            else:
                                G.edge(state, next_states, label=transition)
                                print("next state",next_states)
                                print("state",state)
    return G
file = open("json/NFA.json", "r")
NFA = json.load(file)
NFA = draw_NFA(NFA)
NFA.render('output/NFA', view=True , format='png' ,cleanup=True , quiet=True)



next state S1
state S0
next state S3
state S2


'output/NFA.png'